Code to accompany Machine Learning Recipes #8. We'll write a Decision Tree Classifier, in pure Python. Below each of the methods, I've written a little demo to help explain what it does.

In [2]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [3]:
import json
from sklearn.model_selection import train_test_split

file_name = "phase_2_task_1_transformed_matrix.json"
file_path = file_name
file_handler = open(file_path, 'rb')

transformed_matrix = json.load(file_handler)


print(transformed_matrix)

# configure the gesture map here
gesture_6x = 'vattene'
gesture_1xx = 'Combinato'
gesture_6xx = "D'Accordo"
# 249 - 279
# 559 - 589
config_map = {gesture_6x: [1, 31], gesture_1xx: [249, 279], gesture_6xx: [559, 589]}
result_map = {}
for key in config_map:
    config_range = config_map[key]
    for i in range(config_range[0], config_range[1] + 1):
        result_map[str(i)] = key

input_data = []

for key in transformed_matrix:
    data_list = transformed_matrix[key]
    data_list.append(result_map[key])
    input_data.append(data_list)

training_data, testing_data = train_test_split(input_data, test_size=0.2)


{'21': [-0.0018966752068224982, -0.002660606847216345, -0.002291946946929335, -0.0007371513887472548, -0.001240115704423271], '585': [0.0007822567646397147, 0.0027085795453793845, 0.0027732878949732575, -0.0007403035642362163, -0.0036485107749378103], '584': [-0.0008010307369576662, 0.021951862331384162, -0.012265499771053881, -0.004236161335100189, 0.0032078789855030424], '20': [0.0026117732717216087, 0.006671680684675833, 0.01660718799645481, 0.0007420615926853191, -0.002489936717964901], '22': [0.001829335712193573, 0.0068538449776357315, 0.015945449292484176, 0.0008506717676063426, -0.011884176136246634], '586': [-0.0016376300721305402, -0.002740240315282377, -0.0014576462732701875, 3.0134557628503035e-05, -0.0006346526530073654], '579': [-0.0015947231456111027, -0.0022354396155912218, -0.0021893038641289074, -0.0001522660914792128, -0.002365256010824258], '578': [-0.0005646247459048266, -0.0010193569378912166, 0.0018512864038094338, -0.0011666035902196585, -0.003334338898556224], 

In [4]:
key_size = len(training_data[0]) - 1
# Column labels.
# These are used only to print the tree.
header = []
for i in range(0, key_size):
    header.append('key_' + str(i))
# header = ["color", "diameter", "label"]

header.append("label")

In [5]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [6]:
#######
# Demo:
unique_vals(training_data, 0)
# unique_vals(training_data, 1)
#######

{-0.0037014534800157246,
 -0.0032506173444997043,
 -0.00288864657223625,
 -0.0028049336501283413,
 -0.0027259584188365697,
 -0.002504782098533021,
 -0.002351573451072828,
 -0.002323154716350591,
 -0.0021252232895617113,
 -0.0020746075250712734,
 -0.001988338303575454,
 -0.0018966752068224982,
 -0.0018875287772023917,
 -0.001873547131815951,
 -0.001818140244139399,
 -0.0018054496105487884,
 -0.0017944531059583028,
 -0.0017804550049515925,
 -0.0017787350786056082,
 -0.0017734826357099155,
 -0.0016794824153494152,
 -0.0016376300721305402,
 -0.001628301634444443,
 -0.0015947231456111027,
 -0.0015905006660196167,
 -0.0015761496276298845,
 -0.0015193046856383297,
 -0.0014927100171166522,
 -0.00149148265841141,
 -0.0014641489909135975,
 -0.001428819364656721,
 -0.0014216117463573526,
 -0.0013896109003965638,
 -0.0013793695987594486,
 -0.0013764753790446594,
 -0.001337415766842766,
 -0.0013234896802169723,
 -0.001313593090788391,
 -0.0012468146191337438,
 -0.0012407238235792753,
 -0.0011885020

In [7]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [8]:
#######
# Demo:
class_counts(training_data)
#######

{"D'Accordo": 27, 'vattene': 24, 'Combinato': 23}

In [9]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [10]:
#######
# Demo:
is_numeric(7)
# is_numeric("Red")
#######

True

In [11]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [12]:
#######
# Demo:
# Let's write a question for a numeric attribute
Question(1, 3)

Is key_1 >= 3?

In [13]:
# How about one for a categorical attribute
q = Question(0, 0.0018449816816446488)
q

Is key_0 >= 0.0018449816816446488?

In [14]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.match(example) # this will be true, since the first example is Green.
#######

False

In [15]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [16]:
#######
# Demo:
# Let's partition the training data based on whether rows are greater than equal to a value.
true_rows, false_rows = partition(training_data, Question(0, 0.0013447720368327054))
true_rows

[[0.08642949698486307,
  -0.006059187236759987,
  -0.004250685361693582,
  0.0007380943979362695,
  0.0019542831083719644,
  'vattene'],
 [0.001829335712193573,
  0.0068538449776357315,
  0.015945449292484176,
  0.0008506717676063426,
  -0.011884176136246634,
  'vattene'],
 [0.0013555748478966466,
  0.002367227902076187,
  -0.00022066716555795772,
  -0.0012098734115692808,
  -0.008289928163708472,
  "D'Accordo"],
 [0.0014503134364296365,
  0.004144267011360101,
  -0.0005639858432025724,
  -0.0015783456732535394,
  -0.005488680856443327,
  "D'Accordo"],
 [0.0021983521720932973,
  0.006933323608538396,
  0.03935684318032357,
  -0.043532051847110995,
  0.02396700878539084,
  'vattene'],
 [0.0014595098109878547,
  0.010848964536553632,
  0.020563110415629752,
  0.012041062841321098,
  -0.029624438175940973,
  'vattene'],
 [0.0025199182067354637,
  0.06302118834542153,
  -0.022476728257312808,
  -0.003925808623773344,
  0.008971634087402482,
  'vattene']]

In [17]:
# This will contain everything else.
false_rows
#######

[[-0.001313593090788391,
  -0.0031237397112194986,
  -0.002850337137853562,
  5.333510244441245e-05,
  0.0008185998279696206,
  "D'Accordo"],
 [-0.002504782098533021,
  -0.0044146260279935975,
  -0.004081000477023272,
  -0.00028340627719386806,
  0.0038963958935398554,
  'Combinato'],
 [-0.0020746075250712734,
  -0.0015960670109460419,
  -0.0013435594665926493,
  -0.0004194585101972557,
  -0.0008620855966292441,
  "D'Accordo"],
 [-0.0013896109003965638,
  -0.0033559589031299623,
  -0.0023110041266423465,
  -0.0014961154343451522,
  0.0013498976901776942,
  'Combinato'],
 [-0.00149148265841141,
  -0.002494360830400943,
  -0.0016555190605108475,
  0.0009735820368221595,
  0.0034087251040245646,
  'Combinato'],
 [-0.0010768306292598658,
  -0.0013419477609230518,
  0.0009208407782433579,
  -0.00021685493600226335,
  -0.0015522833184878659,
  "D'Accordo"],
 [-0.00288864657223625,
  -0.00357583848894485,
  -0.005098245531669975,
  -0.00025499942465971115,
  0.003642048517410975,
  'Combinato

In [18]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [19]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['1xx'],
              ['1xx']]
# this will return 0
gini(no_mixing)

0.0

In [20]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['1xx'],
               ['6x']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [21]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['1xx'],
                  ['6x'],
                  ['6xx']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.6666666666666665

In [22]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [23]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.6650840029218408

In [24]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 0.0013447720368327054))
info_gain(true_rows, false_rows, current_uncertainty)

0.02663531969703803

In [25]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,-0.009172534604033632))
info_gain(true_rows, false_rows, current_uncertainty)

0.0

In [26]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(0,0.0013447720368327054))

# Here, the true_rows contain only 'Grapes'.
true_rows

[[0.08642949698486307,
  -0.006059187236759987,
  -0.004250685361693582,
  0.0007380943979362695,
  0.0019542831083719644,
  'vattene'],
 [0.001829335712193573,
  0.0068538449776357315,
  0.015945449292484176,
  0.0008506717676063426,
  -0.011884176136246634,
  'vattene'],
 [0.0013555748478966466,
  0.002367227902076187,
  -0.00022066716555795772,
  -0.0012098734115692808,
  -0.008289928163708472,
  "D'Accordo"],
 [0.0014503134364296365,
  0.004144267011360101,
  -0.0005639858432025724,
  -0.0015783456732535394,
  -0.005488680856443327,
  "D'Accordo"],
 [0.0021983521720932973,
  0.006933323608538396,
  0.03935684318032357,
  -0.043532051847110995,
  0.02396700878539084,
  'vattene'],
 [0.0014595098109878547,
  0.010848964536553632,
  0.020563110415629752,
  0.012041062841321098,
  -0.029624438175940973,
  'vattene'],
 [0.0025199182067354637,
  0.06302118834542153,
  -0.022476728257312808,
  -0.003925808623773344,
  0.008971634087402482,
  'vattene']]

In [27]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[[-0.001313593090788391,
  -0.0031237397112194986,
  -0.002850337137853562,
  5.333510244441245e-05,
  0.0008185998279696206,
  "D'Accordo"],
 [-0.002504782098533021,
  -0.0044146260279935975,
  -0.004081000477023272,
  -0.00028340627719386806,
  0.0038963958935398554,
  'Combinato'],
 [-0.0020746075250712734,
  -0.0015960670109460419,
  -0.0013435594665926493,
  -0.0004194585101972557,
  -0.0008620855966292441,
  "D'Accordo"],
 [-0.0013896109003965638,
  -0.0033559589031299623,
  -0.0023110041266423465,
  -0.0014961154343451522,
  0.0013498976901776942,
  'Combinato'],
 [-0.00149148265841141,
  -0.002494360830400943,
  -0.0016555190605108475,
  0.0009735820368221595,
  0.0034087251040245646,
  'Combinato'],
 [-0.0010768306292598658,
  -0.0013419477609230518,
  0.0009208407782433579,
  -0.00021685493600226335,
  -0.0015522833184878659,
  "D'Accordo"],
 [-0.00288864657223625,
  -0.00357583848894485,
  -0.005098245531669975,
  -0.00025499942465971115,
  0.003642048517410975,
  'Combinato

In [28]:
# # On the other hand, partitioning by Green doesn't help so much.
# true_rows, false_rows = partition(training_data, Question(0,'Green'))
#
# # We've isolated one apple in the true rows.
# true_rows

In [29]:
# But, the false-rows are badly mixed up.
# false_rows
#######

In [30]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [31]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is key_4 >= 0.0010985379880844883?

In [32]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [33]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [34]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [35]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [36]:
my_tree = build_tree(training_data)

In [37]:
print_tree(my_tree)

Is key_4 >= 0.0010985379880844883?
--> True:
  Is key_1 >= -0.000592302341973545?
  --> True:
    Is key_4 >= 0.0022523880555943617?
    --> True:
      Is key_4 >= 0.008971634087402482?
      --> True:
        Predict {'vattene': 3}
      --> False:
        Is key_4 >= 0.0032078789855030424?
        --> True:
          Predict {"D'Accordo": 1}
        --> False:
          Predict {'vattene': 1}
    --> False:
      Predict {'Combinato': 1}
  --> False:
    Is key_0 >= 0.08642949698486307?
    --> True:
      Predict {'vattene': 1}
    --> False:
      Predict {'Combinato': 20}
--> False:
  Is key_2 >= 0.0032742042863826764?
  --> True:
    Predict {'vattene': 6}
  --> False:
    Is key_2 >= -0.0014576462732701875?
    --> True:
      Is key_4 >= -0.0011485417808359704?
      --> True:
        Is key_2 >= -0.0007451888208275377?
        --> True:
          Is key_4 >= -0.0009305033245338677?
          --> True:
            Is key_4 >= -0.000603192889565685?
            --> True:
      

In [38]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [39]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{"D'Accordo": 3}

In [40]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [41]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{"D'Accordo": '100%'}

In [42]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######


{'vattene': '100%'}

In [46]:
total_gestures_in_testing = 0
correctly_guessed = 0
for row in testing_data:
    total_gestures_in_testing += 1
    actual_label = row[-1]
    classified_label = print_leaf(classify(row, my_tree))

    if actual_label in classified_label:
        correctly_guessed += 1

    print ("Actual: %s. Predicted: %s" %(actual_label, classified_label))


print('Number of gestures in the training data: ' + str(len(training_data)))
print('Number of gestures in the testing data: ' + str(len(testing_data)))

print('Accuracy: ' + str(correctly_guessed/total_gestures_in_testing))

Actual: D'Accordo. Predicted: {"D'Accordo": '100%'}
Actual: Combinato. Predicted: {'Combinato': '100%'}
Actual: Combinato. Predicted: {'Combinato': '100%'}
Actual: vattene. Predicted: {'vattene': '100%'}
Actual: D'Accordo. Predicted: {"D'Accordo": '100%'}
Actual: vattene. Predicted: {'vattene': '100%'}
Actual: D'Accordo. Predicted: {"D'Accordo": '100%'}
Actual: vattene. Predicted: {'vattene': '100%'}
Actual: Combinato. Predicted: {"D'Accordo": '100%'}
Actual: Combinato. Predicted: {"D'Accordo": '100%'}
Actual: D'Accordo. Predicted: {"D'Accordo": '100%'}
Actual: Combinato. Predicted: {'vattene': '100%'}
Actual: vattene. Predicted: {'Combinato': '100%'}
Actual: Combinato. Predicted: {'Combinato': '100%'}
Actual: vattene. Predicted: {'vattene': '100%'}
Actual: Combinato. Predicted: {'Combinato': '100%'}
Actual: Combinato. Predicted: {'Combinato': '100%'}
Actual: vattene. Predicted: {'vattene': '100%'}
Actual: vattene. Predicted: {'vattene': '100%'}
Number of gestures in the training data: